In [ ]:
pip install kafka-python

In [ ]:
pip install --upgrade kafka-python

In [9]:
from kafka import KafkaProducer
from kafka import KafkaConsumer, TopicPartition

In [10]:
bootstrap_servers = 'kafka-1:19092,kafka-2:19093,kafka-3:19094'
topic_name = 'msg_delivery_app'

In [11]:
# Acks=0: Fire and forget (no acknowledgment)
producer_acks_0 = KafkaProducer(bootstrap_servers=bootstrap_servers, acks=0)

# Acks=1: Leader acknowledgment
producer_acks_1 = KafkaProducer(bootstrap_servers=bootstrap_servers, acks=1)

# Acks=all: Acknowledgment from all replicas
producer_acks_all = KafkaProducer(bootstrap_servers=bootstrap_servers, acks='all')

In [12]:
print("Producing messages..!")

# Produce messages
for i in range(10):
    message = f'Message {i} acks-test'

    # Encode the message to bytes
    value_bytes = message.encode('utf-8')

    producer_acks_0.send(topic_name, value=value_bytes)
    producer_acks_1.send(topic_name, value=value_bytes)
    producer_acks_all.send(topic_name, value=value_bytes)

Producing messages..!


In [13]:
# Set up Kafka consumer
consumer = KafkaConsumer(
    bootstrap_servers=bootstrap_servers,
    auto_offset_reset='earliest',
    enable_auto_commit=True,
    group_id="",  # Use an empty string to disable group management
    value_deserializer=lambda x: x.decode('utf-8')
)

print("Consumer Configuration:", consumer.config)

Consumer Configuration: {'bootstrap_servers': 'kafka-1:19092,kafka-2:19093,kafka-3:19094', 'client_id': 'kafka-python-2.0.2', 'group_id': '', 'key_deserializer': None, 'value_deserializer': <function <lambda> at 0x7f87817b2200>, 'fetch_max_wait_ms': 500, 'fetch_min_bytes': 1, 'fetch_max_bytes': 52428800, 'max_partition_fetch_bytes': 1048576, 'request_timeout_ms': 305000, 'retry_backoff_ms': 100, 'reconnect_backoff_ms': 50, 'reconnect_backoff_max_ms': 1000, 'max_in_flight_requests_per_connection': 5, 'auto_offset_reset': 'earliest', 'enable_auto_commit': True, 'auto_commit_interval_ms': 5000, 'default_offset_commit_callback': <function KafkaConsumer.<lambda> at 0x7f8781732520>, 'check_crcs': True, 'metadata_max_age_ms': 300000, 'partition_assignment_strategy': (<class 'kafka.coordinator.assignors.range.RangePartitionAssignor'>, <class 'kafka.coordinator.assignors.roundrobin.RoundRobinPartitionAssignor'>), 'max_poll_records': 500, 'max_poll_interval_ms': 300000, 'session_timeout_ms': 100

In [ ]:
# Assign the partitions to the consumer
consumer.assign([TopicPartition(topic_name, 0)])  # Replace '0' with the partition number

# Seek to the end to get the latest offset
consumer.seek_to_end()

# Get the current end offset
end_offset = consumer.position(TopicPartition(topic_name, 0))

# Calculate the offset from which you want to consume (e.g., last 10 messages)
start_offset = max(0, end_offset - 10)

# Seek to the calculated offset
consumer.seek(TopicPartition(topic_name, 0), start_offset)

# Consume messages
for message in consumer:
    print(f"Received message: {message.value}")

# Close the consumer when done
consumer.close()


Received message: Message 0 acks-test
Received message: Message 1 acks-test
Received message: Message 2 acks-test
Received message: Message 3 acks-test
Received message: Message 4 acks-test
Received message: Message 5 acks-test
Received message: Message 6 acks-test
Received message: Message 7 acks-test
Received message: Message 8 acks-test
Received message: Message 9 acks-test


In [ ]:
#producer_acks_0.close()
#producer_acks_1.close()
#producer_acks_all.close()

# Flush and close the producer
producer_acks_0.flush()
producer_acks_0.close()

# Flush and close the producer
producer_acks_1.flush()
producer_acks_1.close()

# Flush and close the producer
producer_acks_all.flush()
producer_acks_all.close()